# Sample Normalisasi Table via Pandas

berdasarkan simulasi disini:
- https://docs.google.com/spreadsheets/d/1DbBQglMIG00EpsVSnrax0A2JE-kbhA40aQkxB4sfRvA/edit?usp=sharing

## UNF

In [61]:
import pandas as pd

df = pd.DataFrame({
    "orderId": [1, 2, 3],
    "customerName": ['Andi', 'Budi', 'Andi'],
    "customerAddress": ['Jl. Mawar No. 1, Jakarta', 'Jl. Melati No. 2, Bogor', 'Jl. Mawar No. 1, Jakarta'],
    "itemsPurchased": ['Laptop, Mouse', 'Keyboard, Monitor, Headset', 'Flashdisk'],
    "totalPrice": ['12000000', '4500000', '150000'],
})

print("hasil UNF")
display(df)

hasil UNF


,orderId,customerName,customerAddress,itemsPurchased,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta","Laptop, Mouse",12000000
1,2,Budi,"Jl. Melati No. 2, Bogor","Keyboard, Monitor, Headset",4500000
2,3,Andi,"Jl. Mawar No. 1, Jakarta",Flashdisk,150000


## 1NF

In [62]:
# karena di kolom itemsPurchased value nya tidak atomic, maka perlu di split supaya setiap barisnya menjadi atomic
# langkah pertama adalah merubah string menjadi list, menggunakan str.split()
df.itemsPurchased = df.itemsPurchased.str.split(',')
print("hasil split")
display(df)

# setelah split(), maka sekarang kolom item list akan dipecah menjadi beberapa baris
# di mana masing2 baris itu hanya 1 item saja, menggunakan df.explode()
df = df.explode(column='itemsPurchased')
print("hasil explode")
display(df)

# berikutnya kita membersihkan setiap value di items purchase dari whitespace, menggunakan str.strip()
df.itemsPurchased = df.itemsPurchased.str.strip()

# kemudian reset index 
df.reset_index(drop=True, inplace=True)

print("hasil akhir 1NF")
display(df)

hasil split


,orderId,customerName,customerAddress,itemsPurchased,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta","[Laptop, Mouse]",12000000
1,2,Budi,"Jl. Melati No. 2, Bogor","[Keyboard, Monitor, Headset]",4500000
2,3,Andi,"Jl. Mawar No. 1, Jakarta",[Flashdisk],150000


hasil explode


,orderId,customerName,customerAddress,itemsPurchased,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta",Laptop,12000000
0,1,Andi,"Jl. Mawar No. 1, Jakarta",Mouse,12000000
1,2,Budi,"Jl. Melati No. 2, Bogor",Keyboard,4500000
1,2,Budi,"Jl. Melati No. 2, Bogor",Monitor,4500000
1,2,Budi,"Jl. Melati No. 2, Bogor",Headset,4500000
2,3,Andi,"Jl. Mawar No. 1, Jakarta",Flashdisk,150000


hasil akhir 1NF


,orderId,customerName,customerAddress,itemsPurchased,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta",Laptop,12000000
1,1,Andi,"Jl. Mawar No. 1, Jakarta",Mouse,12000000
2,2,Budi,"Jl. Melati No. 2, Bogor",Keyboard,4500000
3,2,Budi,"Jl. Melati No. 2, Bogor",Monitor,4500000
4,2,Budi,"Jl. Melati No. 2, Bogor",Headset,4500000
5,3,Andi,"Jl. Mawar No. 1, Jakarta",Flashdisk,150000


## 2NF
- berdasarkan hasil dari simulasi di excel, pada 2NF akan ada 2 table yakni orders dan orderItems

In [63]:
# membuat table orders
orders = df[['orderId', 'customerName', 'customerAddress', 'totalPrice']]
print("table orders")
display(orders)

# membuat table orders agar setiap row nya unique
orders = orders.drop_duplicates(ignore_index=True)
print("table orders with unique rows")
display(orders)

# membuat table orderItems
orderItems = df[['orderId', 'itemsPurchased']]
print("table orderItems")
display(orderItems)

table orders


,orderId,customerName,customerAddress,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta",12000000
1,1,Andi,"Jl. Mawar No. 1, Jakarta",12000000
2,2,Budi,"Jl. Melati No. 2, Bogor",4500000
3,2,Budi,"Jl. Melati No. 2, Bogor",4500000
4,2,Budi,"Jl. Melati No. 2, Bogor",4500000
5,3,Andi,"Jl. Mawar No. 1, Jakarta",150000


table orders with unique rows


,orderId,customerName,customerAddress,totalPrice
0,1,Andi,"Jl. Mawar No. 1, Jakarta",12000000
1,2,Budi,"Jl. Melati No. 2, Bogor",4500000
2,3,Andi,"Jl. Mawar No. 1, Jakarta",150000


table orderItems


,orderId,itemsPurchased
0,1,Laptop
1,1,Mouse
2,2,Keyboard
3,2,Monitor
4,2,Headset
5,3,Flashdisk


## 3NF
- karena di table orders ada transitive dependency, maka harus dihilangkan dengan cara memecah customerName & customerAdress dari orders
- sehingga, sesuai dengan simulasi, akan ada total 3 table yakni, orders, customers, dan orderItems

In [64]:
# membuat table customers
customers = orders[['customerName', 'customerAddress']]

# menghapus duplicate untuk membuat setiap baris menjadi unique
customers = customers.drop_duplicates(ignore_index=True)

print("table customers")
display(customers)

# memperbarui table orders dengan menghapus customerAddress dan tetap menyimpan customerName sebagai reference (FK)
orders = orders.drop(columns=['customerAddress'])

print("table order")
display(orders)

print("table orderItems")
display(orderItems)

table customers


,customerName,customerAddress
0,Andi,"Jl. Mawar No. 1, Jakarta"
1,Budi,"Jl. Melati No. 2, Bogor"


table order


,orderId,customerName,totalPrice
0,1,Andi,12000000
1,2,Budi,4500000
2,3,Andi,150000


table orderItems


,orderId,itemsPurchased
0,1,Laptop
1,1,Mouse
2,2,Keyboard
3,2,Monitor
4,2,Headset
5,3,Flashdisk


## After Normalization

In [65]:
# orders
display(orders)

# orderItems
display(orderItems)

# customers
display(customers)

# extract semua ke csv
orders.to_csv('orders.csv', index=False)
customers.to_csv('customers.csv', index=False)
orderItems.to_csv('orderItems.csv', index=False)

,orderId,customerName,totalPrice
0,1,Andi,12000000
1,2,Budi,4500000
2,3,Andi,150000


,orderId,itemsPurchased
0,1,Laptop
1,1,Mouse
2,2,Keyboard
3,2,Monitor
4,2,Headset
5,3,Flashdisk


,customerName,customerAddress
0,Andi,"Jl. Mawar No. 1, Jakarta"
1,Budi,"Jl. Melati No. 2, Bogor"
